# RSA Cryptografy

When it comes to asymmetric cryptography, RSA (Rivest-Shamir-Adleman) is the default example to explain concepts regarding the topic. The algorithm is one of the first invented (1978) to do so work, rely on a pair of key reversible property of the key to encrypt with one key and roll back to the original message using the other key.

In RSA standard cryptography, two key types are employed, those that will be presented during this document: The RSA public key and the RSA private key and together the private and public key form the key pair needed in the process of encryption and decryption of a message.

The basic principle behind RSA is the multiplication of two factors $p$ and $q$, applying operations on these numbers it is practical to find three very large positive numbers $e$, $d$ and $n$. With these three numbers, it is possible to apply modular exponentiation to reach a strongly cyphered message. To discover the message that originated the cypher text the $p$ and $q$ would be needed, it is possible of factorizing $n$, but $n$ is a large integer, so the process is computationally inviable turning it into a $NP-Complete$ problem.

In [ ]:
import base64
import math
import textwrap

In [ ]:
class LilRSA:
    def __init__(self):
        self.p =    None
        self.q =    None
        self.n =    None
        self.e =    None
        self.d =    None
        self.phi =  None

As mentioned $p$ is the first factor, it is a positive integer. $p$ must be a prime number.

$q$ is the second factor, it is a positive integer. $q$ also must be a prime number.

The function `GenPQ` will set the values for $p$ and $q$, for a normal use they must be random very large numbers for two purposes: So it can work with larger messages, and it is going to be harder to find them again factorizing $n$.

In [ ]:
    def GenPQ(self,d:int):
        if d == 8:
            self.p = 71113279
            self.q = 98327129
        elif d == 16:
            self.p = 4056555933657341
            self.q = 6724511755459679
        elif d == 32:
            self.p = 79307298401562156961148089405447
            self.q = 30824043223426789498907784588361
        elif d == 64:
            self.p = 7102262139724624880661914991820000585426491086958693095823804547
            self.q = 9624173466341420108973789000052858499246519294416603422050306371

The function `GenPair` generates the key pair. It will use other functions to set the value of the variables needed during the process of performing manipulations with RSA.

In [ ]:
    def GenPair(self, d:int=64):

As shown before, we started fining the values for the two factors $p$ and $q$.

In [ ]:
        if self.p == None and self.q == None:
            self.GenPQ(d)

Now it is composed the public key, it consists in two components:

- $e$ is the RSA public expoent, a positive integer. As the number is publicly known there is no problem to be reused multiple times, so $e$ usually has the value $65537$. This public exponent should be a prime number and large, not large as the others, but is large. $65537=2^{16}+1$ and the binary representation is $0b1000000000000001$ there are lots of $0$ turning it easy to work with from the perspective of prossessing cicles.

$$
e = 65537
$$

- $n$ the RSA modulus, a positive integer resulted from the multiplication of $p$ and $q$.

$$
n = p \cdot q
$$

In [ ]:
        self.e = 65537
        self.n = self.p*self.q

$\phi(x)$ represents how many numbers are relatively prime with $x$ so the $gcd$ (greatest common divisor) between $x$ and $y=\{1\leq y < x\}$ must be $1$ for $y$ to be relatively prime with $x$. 

$n$ results from the multiplication of the factors $p$ and $q$, $\phi(n) = \phi(p) \cdot \phi(q)$.  From the fact that $p$ and $q$ are primes, we can suppose that exists $q-1$ numbers relatively prime with $q$, likewise $p$ where $\phi(p)=p-1$:

$$
\phi(n) = (p-1)\cdot (q-1)
$$

In [ ]:
        self.phi = (self.p-1)*(self.q - 1)
        self.FindInversePow()